# AI Agent with OpenAI and Comparision to Amazon Bedrock Agent

For more details, please refer to my blog
[AI Agent: OpenAI AgentSDK vs Amazon Bedrock]()

## OpenAI Agent SDK

A simple demo of creating AI Agent with OpenAI Agent SDK
- Basic Agent
- Multi-agent collaboration
    - handoffs
    - agent as tools

### Set up

In [ ]:
# Virtual Environment
%%python3 -m venv .venv
%source .venv/bin/activate
# Install Package
# OpenAI Agent SDK
%pip install openai-agents
# Amazon Bedrock
%pip install boto3
%pip install botocore

In [ ]:
from agents import Agent, Runner, set_default_openai_key

# Set up OpenAI API KEY
OPENAI_API_KEY="sk-..."
set_default_openai_key(OPENAI_API_KEY)

### Basic agent

In [34]:
pokemon_agent = Agent(
    name="Pokemon agent",
    instructions="You are specialized in pokemons. Your job is to help user understanding pokemon types and choosing the correct pokemon to use in pokemon battle.",
)

In [ ]:
result = await Runner.run(pokemon_agent, "Help me to find a pokemon for battling.")
print(result.final_output)

### Basic HandsOff

In [27]:
fgo_agent = Agent(
    name="Fgo agent",
    instructions="You are an experienced player in Fate/Grand-Order mobile gamp app. Your job is to help user choose servants for different game stages and provide them a good battle strategy.",
)

In [ ]:
game_agent = Agent(
    name="Gameplay agent",
    handoffs=[pokemon_agent, fgo_agent],
    instructions="You are a gameplay supervisor. You determine which agent to use based on the user’s request."
)

In [ ]:
result = await Runner.run(
    game_agent,
    "I am playing FGO and Pokémon card game right now. Help me choose a Pokémon to battle and couple FGO servants to beat the newest stage."
)
print(result.final_output)

#### Agent As Tool

In [28]:
game_agent = Agent(
    name="Gameplay agent",
    tools= [
        pokemon_agent.as_tool(
            tool_name="help_play_pokemon",
            tool_description="help user understanding pokemon types and choosing the correct pokemon to use in pokemon battle."
        ),
        fgo_agent.as_tool(
            tool_name="help_play_fgo",
            tool_description="help user choose servants for different game stages and provide them a good battle strategy."

        )
    ],
    instructions="You are a gameplay supervisor. You use the tool given to you based on the user’s request."
)

In [ ]:
result = await Runner.run(
    game_agent,
    "I am playing FGO and Pokémon card game right now. Help me choose a Pokémon to battle and couple FGO servants to beat the newest stage."
)
print(result.final_output)

Could you provide details on both:

1. The Pokémon you or your opponent might be using.
2. Specifics of the FGO stage you’re taking on (enemy classes, type of stage, any special mechanics, and your available high-level servants).

This will help me give you the best advice!


In [ ]:
# demo of memory not being supported by OpenAI Agent
result = await Runner.run(
    game_agent,
    "Can I get a general recommendation?"
)
print(result.final_output)

Could you please specify the game or area you need recommendations for? For example, Pokémon battles or Fate/Grand Order strategies?


## Bedrock Comparison

A demo on caling bedrock agent to show the difference to OpenAI Agents in terms of **memory**.
- bedrock agent supports memory and is able to continue onto a conversation by providing the same `sessionId`
- To support cross-session memory, first [enable it either from the console or using the SDK](https://docs.aws.amazon.com/bedrock/latest/userguide/agents-configure-memory.html), and then pass in the same `memoryId` when call `invoke_agent`

<br>

For details on creating bedrock agents, please refer to my articles
- [Create Our Own AI Agent with Amazon Bedrock](https://levelup.gitconnected.com/create-our-own-ai-agent-with-amazon-bedrock-96060e4eca43)
= [Amazon Bedrock: Multi-Agent Collaborations](https://medium.com/@itsuki.enjoy/amazon-bedrock-multi-agent-collaborations-c914b27ad5f2)

In [ ]:
import boto3
import json

agent_id = "..."
agent_alias_id = "..."
# By using the same session ID, we can continue the same conversation with an agent
session_id = "00000001"

In [ ]:
runtime_client = boto3.client(
    service_name="bedrock-agent-runtime",
    region_name="ap-northeast-1",
)

In [ ]:
# For processing EventStream
def process_stream(event_stream):
    final_answer = None
    try:
        for event in event_stream:
            if 'chunk' in event:
                data = event['chunk']['bytes']
                final_answer = data.decode('utf8')
                print(f"Final answer ->\n{final_answer}")
                end_event_received = True
            elif 'trace' in event:
                print(json.dumps(event['trace'], indent=2))
            else:
                raise Exception("unexpected event.", event)
    except Exception as e:
        raise Exception("unexpected event.",e)

    return final_answer

In [ ]:
# https://boto3.amazonaws.com/v1/documentation/api/latest/reference/services/index.html
response = runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    inputText="I am playing FGO and Pokémon card game right now. Help me choose a Pokémon to battle and couple FGO servants to beat the newest stage.",
)

In [ ]:
answer = process_stream(response.get("completion"))

Final answer ->
Thank you for your question about both Pokémon card game and Fate/Grand Order (FGO). I've gathered some advice for you on both games.

For Pokémon card game:
When choosing a Pokémon for battle, consider these factors:
1. Type advantages: Select Pokémon with types strong against your opponent's Pokémon.
2. Stats and abilities: Look for Pokémon with high relevant stats and useful abilities.
3. Movesets: Choose Pokémon with diverse and powerful moves.
4. Team balance: Ensure a good mix of types and roles in your team.
5. Experience level: Higher-level Pokémon generally perform better.
6. Special abilities: Some Pokémon have unique abilities that can give you an edge.

For Fate/Grand Order:
When selecting servants for a new stage, keep in mind:
1. Class advantage: Choose servants with advantage against the stage's enemies.
2. Team balance: Include a mix of damage dealers, support, and defensive servants.
3. Stage gimmicks: Consider any special mechanics that might require s

In [19]:
response = runtime_client.invoke_agent(
    agentId=agent_id,
    agentAliasId=agent_alias_id,
    sessionId=session_id,
    inputText="Can I get a general recommendation?",
)

In [ ]:
answer = process_stream(response.get("completion"))

Final answer ->
Certainly! I'd be happy to provide you with some general recommendations for both Pokémon and Fate/Grand Order.

For Pokémon:
A versatile Pokémon that's generally good in many battle situations is Charizard. Here's why:
1. Dual typing (Fire/Flying) gives it advantages against multiple types.
2. Strong Special Attack and Speed stats make it a powerful offensive Pokémon.
3. Diverse movepool allows it to learn a variety of moves for different situations.
4. Mega Evolution options (where available) increase its versatility.
5. Useful abilities (Blaze or Solar Power) can provide additional power boosts.

Charizard's combination of power, speed, and type coverage makes it a solid choice for many battle scenarios. However, be cautious of its weaknesses to Rock, Electric, and Water types.

For Fate/Grand Order:
Here are some versatile servants that are generally good in many situations, including newer stages:

1. Zhuge Liang (Lord El-Melloi II): An excellent support Caster who